<a href="https://colab.research.google.com/github/purbid/NLP-projects/blob/main/entailment_example_with_research_paper_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [2]:
import string
import random
from collections import defaultdict
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
encoder = AutoModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/entialment_model_example/entailment_multi_class_ds.csv')

In [55]:
df.head()

,Title,Conference
0,Innovation in Database Management: Computer Sc...,VLDB
1,High performance prime field multiplication fo...,ISCAS
2,enchanted scissors: a scissor interface for su...,SIGGRAPH
3,Detection of channel degradation attack by Int...,INFOCOM
4,Pinning a Complex Network through the Betweenn...,ISCAS


In [6]:
descriptions_dict = {'VLDB':'This is storage databases VLDB',
  'ISCAS' : 'This is circuits and systems ISCAS',
  'SIGGRAPH':'This is graohics and interactive design SIGGRAPH',
  'INFOCOM' : 'This is networks INFOCOM',
  'WWW' : 'This is query and graph WWW'}

In [56]:
all_x, all_y, entailment_texts = [], [], []

for itr, row in df.iterrows():

  all_x.append(row['Title'])
  all_x.append(row['Title'])

  entailment_texts.append(descriptions_dict[row['Conference']])
  all_y.append(1)

  candidate_list = []
  
  for desc in descriptions_dict:
    if desc.lower()!= row['Conference'].lower():
      candidate_list.append(desc)
  negative_label = random.choice(candidate_list)
  entailment_texts.append(descriptions_dict[negative_label])
  all_y.append(0)

In [58]:
# all_x
# all_y
# entailment_texts

for x, y, ent in zip (all_x[:10], all_y[:10], entailment_texts[:10]):
  print(x+'\t'+ent+ '\t' + str(y))



Innovation in Database Management: Computer Science vs. Engineering.	This is storage databases VLDB	1
Innovation in Database Management: Computer Science vs. Engineering.	This is circuits and systems ISCAS	0
High performance prime field multiplication for GPU.	This is circuits and systems ISCAS	1
High performance prime field multiplication for GPU.	This is query and graph WWW	0
enchanted scissors: a scissor interface for support in cutting and interactive fabrication.	This is graohics and interactive design SIGGRAPH	1
enchanted scissors: a scissor interface for support in cutting and interactive fabrication.	This is circuits and systems ISCAS	0
Detection of channel degradation attack by Intermediary Node in Linear Networks.	This is networks INFOCOM	1
Detection of channel degradation attack by Intermediary Node in Linear Networks.	This is graohics and interactive design SIGGRAPH	0
Pinning a Complex Network through the Betweenness Centrality Strategy.	This is circuits and systems ISCAS	1

In [59]:
print(len(all_x), len(all_y), len(entailment_texts))

5014 5014 5014


In [46]:
train_data_size = int(0.5*len(all_x))
print('training data size is  {} '.format(train_data_size))

training data size is  2507 


In [47]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

In [48]:
train_encodings = tokenizer(all_x[:train_data_size], entailment_texts[:train_data_size], max_length = 400, pad_to_max_length=True, truncation=True)
val_encodings = tokenizer(all_x[train_data_size:], entailment_texts[train_data_size:], max_length = 400, pad_to_max_length=True, truncation=True)

class RhetDataSet(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = RhetDataSet(train_encodings, all_y[:train_data_size])
val_dataset = RhetDataSet(val_encodings, all_y[train_data_size:])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [49]:
###### let's see how the training data looks like ###
for i in range(6):
  print(str(tokenizer.convert_ids_to_tokens(train_dataset[i]['input_ids'])).split('[PAD]')[0] + '\t' + str(train_dataset[i]['labels']))

['[CLS]', 'innovation', 'in', 'database', 'management', ':', 'computer', 'science', 'vs', '.', 'engineering', '.', '[SEP]', 'this', 'is', 'storage', 'databases', 'v', '##ld', '##b', '[SEP]', '	tensor(1)
['[CLS]', 'innovation', 'in', 'database', 'management', ':', 'computer', 'science', 'vs', '.', 'engineering', '.', '[SEP]', 'this', 'is', 'query', 'and', 'graph', 'www', '[SEP]', '	tensor(0)
['[CLS]', 'high', 'performance', 'prime', 'field', 'multiplication', 'for', 'gp', '##u', '.', '[SEP]', 'this', 'is', 'circuits', 'and', 'systems', 'is', '##cas', '[SEP]', '	tensor(1)
['[CLS]', 'high', 'performance', 'prime', 'field', 'multiplication', 'for', 'gp', '##u', '.', '[SEP]', 'this', 'is', 'networks', 'info', '##com', '[SEP]', '	tensor(0)
['[CLS]', 'enchanted', 'scissors', ':', 'a', 'sci', '##sso', '##r', 'interface', 'for', 'support', 'in', 'cutting', 'and', 'interactive', 'fabrication', '.', '[SEP]', 'this', 'is', 'gr', '##ao', '##hic', '##s', 'and', 'interactive', 'design', 'si', '##gg',

In [50]:
device = 'cuda'
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 2).to(device)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.21.1",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9c169103d7e5a

In [51]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/IIT_law_ai/semantic_segmentation/sentence_level_entailment/trained_models/',          # output directory
    num_train_epochs=5,              # total number of training epochs
    evaluation_strategy= "epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='/content/drive/MyDrive/IIT_law_ai/semantic_segmentation/sentence_level_entailment/trained_models/logs/',            # directory for storing logs
    logging_steps=10,
    load_best_model_at_end=True
)
trainer = Trainer(
    model=model.to(device),                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [52]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2507
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1570


Epoch,Training Loss,Validation Loss
1,0.538700,0.478679
2,0.505600,0.359245
3,0.546100,0.438924
4,0.242800,0.504818
5,0.088000,0.561743


***** Running Evaluation *****
  Num examples = 2507
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/IIT_law_ai/semantic_segmentation/sentence_level_entailment/trained_models/checkpoint-314
Configuration saved in /content/drive/MyDrive/IIT_law_ai/semantic_segmentation/sentence_level_entailment/trained_models/checkpoint-314/config.json
Model weights saved in /content/drive/MyDrive/IIT_law_ai/semantic_segmentation/sentence_level_entailment/trained_models/checkpoint-314/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2507
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/IIT_law_ai/semantic_segmentation/sentence_level_entailment/trained_models/checkpoint-628
Configuration saved in /content/drive/MyDrive/IIT_law_ai/semantic_segmentation/sentence_level_entailment/trained_models/checkpoint-628/config.json
Model weights saved in /content/drive/MyDrive/IIT_law_ai/semantic_segmentation/sentence_level_entailment/trained_models/checkpoint-628/

TrainOutput(global_step=1570, training_loss=0.3562093260371761, metrics={'train_runtime': 393.4918, 'train_samples_per_second': 31.856, 'train_steps_per_second': 3.99, 'total_flos': 1297249095432000.0, 'train_loss': 0.3562093260371761, 'epoch': 5.0})

In [53]:
from sklearn import metrics
import numpy as np
predictions_full, labels, _ = trainer.predict(val_dataset)
predictions = np.argmax(predictions_full, axis = 1)
print(metrics.classification_report(labels, predictions))

***** Running Prediction *****
  Num examples = 2507
  Batch size = 8


              precision    recall  f1-score   support

           0       0.87      0.83      0.85      1254
           1       0.84      0.88      0.86      1253

    accuracy                           0.86      2507
   macro avg       0.86      0.86      0.86      2507
weighted avg       0.86      0.86      0.86      2507



In [81]:
from sklearn import metrics

def add_negative_examples_for_validation(input_sentence, entailment_text):
    curr_input_all, curr_labels_all, curr_entailment_texts_all = [],  [], [] #1, 1, 1
    real_label = entailment_text.split(' ')[-1]
    for description in descriptions_dict :
      if description.lower()!=real_label.lower():
        curr_entailment_texts_all.append(descriptions_dict[description])
        curr_labels_all.append(0)
        curr_input_all.append(input_sentence)

    random_index_for_true_text = random.choice(range(len(descriptions_dict)-1))
    # print(random_index_for_true_text)
    curr_input_all.insert(random_index_for_true_text, input_sentence)
    curr_labels_all.insert(random_index_for_true_text, 1)
    curr_entailment_texts_all.insert(random_index_for_true_text, entailment_text)

    return curr_input_all, curr_labels_all, curr_entailment_texts_all


def pick_sentence_with_highest_probability(probs_tensor):


  probabilities_sub_tensor = probs_tensor #prospective_sentences*n_tags
  pos = probabilities_sub_tensor.max(dim=1).values.argmax()
  # print(probs_tensor)
  preds_sublist = probs_tensor.argmax(dim = 1)
  # print(preds_sublist)



  idx_list = []
  
  for pred in preds_sublist:
    if pred == 0:
      idx_list.append([False]*2)
    else:
      idx_list.append([True]*2)
  condition  = torch.tensor(idx_list)
  
  probabilities_sub_tensor = probabilities_sub_tensor.where(condition.to('cuda'), torch.tensor(-1000.0).to('cuda'))  
  probabilities_sub_tensor = probabilities_sub_tensor.max(dim=1)
  pos = probabilities_sub_tensor.values.argmax() 

  
  return pos

def validate_examples(list_of_sentences = [], list_of_labels = [], list_of_entailment_texts = []):
 
  real_labels, pred_labels = [], []
  print('validating on model, output will be label wise F1') #validation
  
  assert len(list_of_sentences) == len(list_of_labels), "Length of labels doesn't match inputs" 

  model.eval()
  print('total sentences are : {}'.format(len(list_of_sentences)//2))
  sent_num = 0
  for sent, label, entailment_text in zip(list_of_sentences, list_of_labels, list_of_entailment_texts):
    if sent_num%100==0:
      print(sent_num)
    sent_num+=1
    if label == 0:
      continue # we don't want negative labels as of now
    else:
      curr_sentence_all_set, all_labels_set, curr_entailment_texts_all = add_negative_examples_for_validation(sent, entailment_text)
      # for a, b, c in zip(curr_sentence_all_set, curr_entailment_texts_all, all_labels_set):
      #   print(a+'\t'+b+'\t'+str(c))
      # break
      # Dissemination of Models over Time-Varying Data., This is storage databases VLDB
      
      validation_input = tokenizer(curr_sentence_all_set, curr_entailment_texts_all, max_length = 400, pad_to_max_length=True, truncation=True, return_tensors='pt').to(device)
      with torch.no_grad():
        class_scores = model(**validation_input)
      # tensors of shape 5*2   
      pred_label_pos = pick_sentence_with_highest_probability(class_scores.logits)
      real_labels.append(entailment_text.split(' ')[-1])
      pred_labels.append(curr_entailment_texts_all[pred_label_pos].split(" ")[-1])
      
  
  print(metrics.classification_report(real_labels, pred_labels))




def infer_example(list_of_sentences = []):
 
  real_labels, pred_labels = [], []
  print('validating on model, output will be label wise F1') #validation
  

  model.eval()
  sent_num = 0
  for sent in list_of_sentences:
    
    sent_num+=1
    curr_sentence_all_set, _, curr_entailment_texts_all = add_negative_examples_for_validation(sent, entailment_text='')
   
    
    validation_input = tokenizer(curr_sentence_all_set, curr_entailment_texts_all, max_length = 400, pad_to_max_length=True, truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
      class_scores = model(**validation_input)
    # tensors of shape 5*2   
    pred_label_pos = pick_sentence_with_highest_probability(class_scores.logits)
    print(curr_entailment_texts_all)
    print(class_scores.logits)
    # real_labels.append(entailment_text.split(' ')[-1])
    print(curr_entailment_texts_all[pred_label_pos].split(" ")[-1])
    # pred_labels.append(curr_entailment_texts_all[pred_label_pos].split(" ")[-1])
      
  

probs = validate_examples(all_x[train_data_size:train_data_size+500], all_y[train_data_size:train_data_size+500], entailment_texts[train_data_size:train_data_size+500])
probs = infer_example([''])  


validating on model, output will be label wise F1
['This is storage databases VLDB', 'This is circuits and systems ISCAS', 'This is graohics and interactive design SIGGRAPH', '', 'This is networks INFOCOM', 'This is query and graph WWW']
tensor([[-1.3913,  1.2024],
        [-0.2737,  0.2637],
        [ 1.6128, -1.6919],
        [ 0.0789, -0.0815],
        [-0.6725,  0.5155],
        [-0.0874, -0.1658]], device='cuda:0')
VLDB


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
